In [ ]:
from IPython.display import clear_output

# For Installing Comfy UI
def install_comfy_ui(checkout_version = "37d620a6b85f61b824363ed8170db373726ca45a"):
    # Start to install Comfy UI and its Manager
    %cd {working_dir}
    !git clone https://github.com/comfyanonymous/ComfyUI.git
    %cd ComfyUI
    # Old Commit. March Version
    # !git checkout 7fc3ccdcc2fb1f20c4b7dd4aca374db952fd66df
    # New Commit 28 July 2025
    # !git checkout 78672d0ee6d20d8269f324474643e5cc00f1c348
    !git checkout {checkout_version}
    # Install the necessary requirements
    !{pip} install -r requirements.txt

    # clear_output()

    !mkdir /tmp/models
    !mkdir /tmp/models/checkpoints
    !mkdir /tmp/models/diffusion_models
    !mkdir /tmp/models/clip
    !mkdir /tmp/models/clip_vision
    !mkdir /tmp/models/ipadapter
    !mkdir /tmp/models/text_encoders
    !mkdir /tmp/models/vae
    !mkdir /tmp/models/unet

    # Require the ip adapter folder
    !mkdir {working_dir}/ComfyUI/models/ipadapter

    # Remove the following two lines to keep models in permanent storage
    !rm -rf {working_dir}/ComfyUI/models/checkpoints
    !rm -rf {working_dir}/ComfyUI/models/diffusion_models
    !rm -rf {working_dir}/ComfyUI/models/clip
    !rm -rf {working_dir}/ComfyUI/models/clip_vision
    !rm -rf {working_dir}/ComfyUI/models/ipadapter
    !rm -rf {working_dir}/ComfyUI/models/text_encoders
    !rm -rf {working_dir}/ComfyUI/models/vae
    !rm -rf {working_dir}/ComfyUI/models/unet

    !ln -s /tmp/models/checkpoints {working_dir}/ComfyUI/models/checkpoints
    !ln -s /tmp/models/diffusion_models {working_dir}/ComfyUI/models/diffusion_models
    !ln -s /tmp/models/clip {working_dir}/ComfyUI/models/clip
    !ln -s /tmp/models/clip_vision {working_dir}/ComfyUI/models/clip_vision
    !ln -s /tmp/models/ipadapter {working_dir}/ComfyUI/models/ipadapter
    !ln -s /tmp/models/text_encoders {working_dir}/ComfyUI/models/text_encoders
    !ln -s /tmp/models/vae {working_dir}/ComfyUI/models/vae
    !ln -s /tmp/models/unet {working_dir}/ComfyUI/models/unet

    # Additional installation to use
    # !{pip} install pillow==10.2.0 insightface onnxruntime onnxruntime-gpu
    # !{pip} install matplotlib-inline

    # Second GPU offload
    %cd {working_dir}/ComfyUI/custom_nodes
    !wget https://gist.githubusercontent.com/city96/30743dfdfe129b331b5676a79c3a8a39/raw/ecb4f6f5202c20ea723186c93da308212ba04cfb/ComfyBootlegOffload.py

    # Return to the working directory
    %cd {working_dir}

    ## Installing standard Nodes
    # Loading ComfyUI-Manager
    git_custom_node("https://github.com/Comfy-Org/ComfyUI-Manager")
    # For installing custom nodes
    git_custom_node("https://github.com/Kosinkadink/ComfyUI-VideoHelperSuite")
    git_custom_node("https://github.com/kijai/ComfyUI-KJNodes")
    # git_custom_node("https://github.com/yolain/ComfyUI-Easy-Use","717092a3ceb51c474b5b3f77fc188979f0db9d67")
    git_custom_node("https://github.com/rgthree/rgthree-comfy","944d5353a1b0a668f40844018c3dc956b95a67d7")

    # for loading GGUF model
    git_custom_node("https://github.com/city96/ComfyUI-GGUF")
    # for multi GPU
    git_custom_node("https://github.com/pollockjj/ComfyUI-MultiGPU")

    # for interpolation
    # git_custom_node("https://github.com/Fannovel16/ComfyUI-Frame-Interpolation")
    git_custom_node("https://github.com/kijai/ComfyUI-GIMM-VFI")

    # clear_output()
    !echo "ComfyUI Installed Successfully"

In [ ]:
# This script allows comfy-ui to run in the background

import subprocess
import threading
from tqdm.auto import tqdm
import re

# For Running Comfy UI in the background
def run_comfy_ui_background():
    # Setting up the command. Note that python environment should come from env
    command = [python, f"{working_dir}/ComfyUI/main.py", "--dont-print-server"]

    # The Thread that Run ComfyUI
    main_process = subprocess.Popen(command,stdout=subprocess.PIPE,stderr=subprocess.STDOUT,text=True,bufsize=1)

    def output_reader(process):
        pbar = None
        progress_pattern = re.compile(r'(\d+)%\|.*?\| (\d+)/(\d+)')

        for line in process.stdout:
            match = progress_pattern.search(line)
            if match:
                percentage, current, total = map(int, match.groups())
                if pbar is None:
                    pbar = tqdm(total=total, unit='it')
                pbar.n = current
                pbar.refresh()
            else:
                print(f"main.py: {line}", end='', flush=True)

        if pbar:
            pbar.close()

    threading.Thread(target=output_reader, args=(main_process,), daemon=True).start()
    return main_process